<a href="https://colab.research.google.com/github/nyp-sit/iti107-2024S2/blob/main/session-3/yolo8_custom_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create an object detection dataset

the dataset is relate to classification of fire fighting equipments and safety deviec such as fire extinguisher, helmelt,fire signage with total around 100 pictures, and one video which relate to sale product of fire fighting equipments.

the pictures have been anoted by roboflow to lable with 'helmet',fire extighuisher and fire signage.

the images have been download to dataset folder per Ultralytics training set requirments.
### Raw Image Dataset


There are total of 100 images.
the images have been divided into both training and validation set.
 (e.g. 80%-20%, i.e. 59 images for train, and 15 for test).


### Download Annotated Dataset

To save you time for this lab, you can download a pre-annotated balloon dataset [here](https://github.com/nyp-sit/iti107-2024S2/raw/refs/heads/main/data/balloon_annotated_dataset.zip).

We download and unzip to the directory called `datasets`



In [2]:
# Install GitPython if not already installed

import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

!pip install gitpython
# Clone the repository using GitPython
import git
repo_url = "https://github.com/Marshmellow589/iti107-assignment-datasets.git"
repo_dir = "datasets"
git.Repo.clone_from(repo_url, repo_dir)
print(f"Repository cloned to {repo_dir}")

Repository cloned to datasets


In [3]:
%%capture
# import platform 
# !pip install --upgrade ultralytics
# !!pip install ultralytics[default]
!pip install comet_ml

## Training the Model

YOLOv8 comes with different sizes of pretrained models: yolov8n, yolov8s, .... The differs in terms of their sizes, inference speeds and precision:

<img src="https://github.com/nyp-sit/iti107-2024S2/blob/main/assets/yolo-models.png?raw=true" width="70%"/>


We will use the small pretrained model yolo8s and finetune it on our custom dataset.


### Setup the logging

Ultralytics support logging to wandb, comet.ml and tensorboard, out of the box. Here we only enable wandb.

You need to create an account at [wandb](https://wandb.ai) and get the API key from https://wandb.ai/authorize.


In [4]:
import platform  # Ensure the 'platform' module is imported
import sys  # Used to reload modules
import ultralytics  # Import the library

# Force a reload of ultralytics
sys.modules.pop('ultralytics', None)  # Remove from cache
sys.modules.pop('ultralytics.yolo.v8', None)  # Remove a specific submodule
import ultralytics  # Import ultralytics again to reload

from ultralytics import YOLO
from ultralytics import settings

settings.update({"wandb": True,
                 "comet.ml": False,
                 "tensorboard": False})

WARNING  Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\yinbo\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


### Training

We specify the path to data.yaml file, and train with a batch size of 15, and we also save the checkpoint at each epoch (save_period=1). We assume here you are connected to a GPU, hence we can specify the device to use as `device=0` to select the first GPU.  We specify the project name as balloon, this will create a folder called `balloon` to store the weights and various training artifacts such as F1, PR curves, confusion matrics, training results (loss, mAP, etc).

For a complete listing of train settings, you can see [here](https://docs.ultralytics.com/modes/train/#train-settings).

You can also specify the type of data [augmentation](https://docs.ultralytics.com/modes/train/#augmentation-settings-and-hyperparameters)  you want as part of the train pipeline.

You can monitor your training progress at wandb (the link is given in the train output below)


In [5]:
# Define data augmentation parameters 
augmentation_params = {'hsv_h': 0.015, # HSV-Hue augmentation 
                       'hsv_s': 0.7, # HSV-Saturation augmentation 
                       'hsv_v': 0.4, # HSV-Value augmentation 
                       'degrees': 0.0, # Image rotation (+/- deg) 
                       'translate': 0.1, # Image translation (+/- fraction) 
                       'scale': 0.5, # Image scale (+/- gain) 
                       # 'shear': 0.0, # Image shear (+/- deg) 
                       # 'perspective': 0.0, # Image perspective (+/- fraction), range 0-0.001 
                       'flipud': 0.0, # Flip up-down (probability) 
                       # 'fliplr': 0.5, # Flip left-right (probability) 
                       # 'mosaic': 1.0, # Mosaic augmentation (probability) 
                       'mixup': 0.0, # Mixup augmentation (probability) 
                       # 'copy_paste': 0.0 # Copy-paste augmentation (probability) 
                      }

In [6]:
from ultralytics import YOLO
from ultralytics import settings

model = YOLO("yolov8s.pt")  # Load a pre-trained YOLO model
result = model.train(data="datasets/data.yaml",
                     epochs=30,
                     save_period=1,
                     batch=16,
                     device='cpu',
                     project='iti107_assignment',
                     **augmentation_params, # Apply data augmentation parameters,
                     plots=True)

100%|█████████████████████████████████████████████████████████████████████████████| 21.5M/21.5M [00:01<00:00, 20.2MB/s]


Ultralytics 8.3.46  Python-3.8.18 torch-2.4.1+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
engine\trainer: task=detect, mode=train, model=yolov8s.pt, data=datasets/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=cpu, workers=8, project=iti107_assignment, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

E:\Anaconda\envs\pythontest\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

train: Scanning E:\Anaconda\envs\00.NYP course\itit107-spdc2\assignment-iti107\datasets\train\labels... 41 images, 0 ba

...


30 epochs completed in 0.332 hours.
Optimizer stripped from iti107_assignment\train\weights\last.pt, 22.5MB
Optimizer stripped from iti107_assignment\train\weights\best.pt, 22.5MB

Validating iti107_assignment\train\weights\best.pt...
Ultralytics 8.3.46  Python-3.8.18 torch-2.4.1+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Model summary (fused): 168 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<0


                   all         10         23      0.611      0.823      0.825      0.528
   extinguisher-helmet          2          3          1      0.601      0.705      0.626
     fire extinguisher          4          7      0.599          1      0.924      0.552
          fire signage          5          6      0.365      0.833      0.811      0.486
                helmet          3          7      0.479      0.857      0.861      0.447
Speed: 3.0ms preprocess, 212.6ms inference, 0.0ms loss, 4.7ms postprocess per image
Results saved to iti107_assignment\train


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml OfflineExperiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : train
COMET INFO:     url                   : [OfflineExperiment will get URL after upload]
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     lr/pg0 [61]               : (2.5e-05, 0.0002966875)
COMET INFO:     lr/pg1 [61]               : (2.5e-05, 0.0002966875)
COMET INFO:     lr/pg2 [61]               : (2.5e-05, 0.0002966875)
COMET INFO:     metrics/mAP50(B) [31]     : (0.1253, 0.8253728317659352)
COMET INFO:     metrics/mAP50-95(B) [31]  : (0.07206, 0.53013)
COMET INFO:     metrics/mAP50-95B [31]    : (0.07206, 0.53013)
COMET INFO:     metrics/mAP50B [31]       : (0.1253, 0.8253728317659352)
COMET INFO:     metrics/precision(B) [31]

MLflow: results logged to runs\mlflow
MLflow: disable with 'yolo settings mlflow=False'


You can see the various graphs in  dashboard, for example:

*metrics*

<img src="https://github.com/nyp-sit/iti107-2024S2/blob/main/assets/wandb-metrics.png?raw=true"/>

*Train and validation loss*

<img src="https://github.com/nyp-sit/iti107-2024S2/blob/main/assets/wandb-loss.png?raw=true"/>

You can go to the folder `iti107_assignment-->train-->weights` and you will files like epoch0.pt, epoch1.pt, .... and also best.pt.
The epoch0.pt, epoch1.pt are the checkpoints that are saved every period (in our case, we specify period as 1 epoch).  The best.pt contains the best checkpoint.

We can run the best model (using the best checkpoint) against the validation dataset to see the overall model performance on validation set.  

You should see around `0.88` for `mAP50`, and `0.78` for `mAP50-95`.

In [8]:
from ultralytics import YOLO

model = YOLO("iti107_assignment/train/weights/best.pt")
validation_results = model.val(data="datasets/data.yaml", device="cpu")

Ultralytics 8.3.46  Python-3.8.18 torch-2.4.1+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
Model summary (fused): 168 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning E:\Anaconda\envs\00.NYP course\itit107-spdc2\assignment-iti107\datasets\valid\labels.cache... 10 images, 
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<0


                   all         10         23      0.611      0.823      0.825      0.528
   extinguisher-helmet          2          3          1      0.601      0.705      0.626
     fire extinguisher          4          7      0.599          1      0.924      0.552
          fire signage          5          6      0.365      0.833      0.811      0.486
                helmet          3          7      0.479      0.857      0.861      0.447
Speed: 2.2ms preprocess, 185.9ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs\detect\val


## Export and Deployment

Your model is in pytorch format (.pt). You can export the model to various format, e.g. TorchScript, ONNX, OpenVINO, TensorRT, etc. depending on your use case, and deployment platform (e.g. CPU or GPU, etc)

You can see the list of [supported formats](https://docs.ultralytics.com/modes/export/#export-formats)  and the option they support in terms of further optimization (such as imagesize, int8, half-precision, etc) in the ultralytics site.

Ultralytics provide a utility function to benchmark your model using different supported formats automatically. You can uncomment the code in the following code cell to see the benchmark result. If you are benchmarking for CPU only, the change the `device=0` to `device='cpu'`.  

**Beware: it will take quite a while to complete the benchmark**

In [ ]:
# from ultralytics.utils.benchmarks import benchmark

# # Benchmark on GPU (device=0 means the 1st GPU device)
# benchmark(model="balloon/train/weights/best.pt", data="datasets/data.yaml", imgsz=640, half=False, device=0)

In [9]:

!pip install openvino
!pip install nncf==2.13.0


In the following code, we export it as OpenVINO. OpenVINO is optimized for inference on Intel CPUs and since we will use the model later on to do inference on local Windows machine (which runs Intel chip), we will export it as OpenVINO format. We also specify using int8 quantization, which results in faster inference, at the cost of accuracy.

For more information on OpenVINO, go to the [official documentation](https://docs.openvino.ai/2024/index.html).

After export, you can find the openvino model in `iti107_assignment\train\weights\best_openvino_model` directory.

In [11]:

model = YOLO("iti107_assignment/train/weights/best.pt")
exported_path = model.export(format="openvino", int8=True)

Ultralytics 8.3.46  Python-3.8.18 torch-2.4.1+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
WARNING  INT8 export requires a missing 'data' arg for calibration. Using default 'data=coco8.yaml'.
Model summary (fused): 168 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs

PyTorch: starting from 'iti107_assignment\train\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 8, 8400) (21.5 MB)
requirements: Ultralytics requirement ['openvino>=2024.5.0'] not found, attempting AutoUpdate...
Retry 1/2 failed: Command 'pip install --no-cache-dir "openvino>=2024.5.0" ' returned non-zero exit status 1.
Retry 2/2 failed: Command 'pip install --no-cache-dir "openvino>=2024.5.0" ' returned non-zero exit status 1.
requirements:  Command 'pip install --no-cache-dir "openvino>=2024.5.0" ' returned non-zero exit status 1.

OpenVINO: starting export with openvino 2024.0.0-14509-34caeefd078-releases/2024/0...
requirements: Ultralytics requirement ['nncf>=2.14.0'] not 

100%|███████████████████████████████████████████████████████████████████████████████| 433k/433k [00:00<00:00, 3.74MB/s]
Unzipping E:\Anaconda\envs\00.NYP course\itit107-spdc2\assignment-iti107\datasets\coco8.zip to E:\Anaconda\envs\00.NYP 

Dataset download success  (3.1s), saved to E:\Anaconda\envs\00.NYP course\itit107-spdc2\assignment-iti107\datasets




Scanning E:\Anaconda\envs\00.NYP course\itit107-spdc2\assignment-iti107\datasets\coco8\labels\val... 4 images, 0 backgr

New cache created: E:\Anaconda\envs\00.NYP course\itit107-spdc2\assignment-iti107\datasets\coco8\labels\val.cache
OpenVINO: WARNING  >300 images recommended for INT8 calibration, found 4 images.


INFO:nncf:25 ignored nodes were found by patterns in the NNCFGraph
INFO:nncf:1 ignored nodes were found by types in the NNCFGraph
INFO:nncf:Not adding activation input quantizer for operation: 162 __module.model.22.dfl/aten::view/Reshape
INFO:nncf:Not adding activation input quantizer for operation: 163 __module.model.22/aten::sigmoid/Sigmoid
INFO:nncf:Not adding activation input quantizer for operation: 173 __module.model.22.dfl/aten::transpose/Transpose
INFO:nncf:Not adding activation input quantizer for operation: 182 __module.model.22.dfl/aten::softmax/Softmax
INFO:nncf:Not adding activation input quantizer for operation: 190 __module.model.22.dfl.conv/aten::_convolution/Convolution
INFO:nncf:Not adding activation input quantizer for operation: 189 __module.model.22.dfl/aten::view/Reshape_1
INFO:nncf:Not adding activation input quantizer for operation: 203 __module.model.22/aten::sub/Subtract
INFO:nncf:Not adding activation input quantizer for operation: 204 __module.model.22/aten:

Output()

Output()

OpenVINO: export success  36.1s, saved as 'iti107_assignment\train\weights\best_int8_openvino_model\' (11.2 MB)

Export complete (36.8s)
Results saved to E:\Anaconda\envs\00.NYP course\itit107-spdc2\assignment-iti107\iti107_assignment\train\weights
Predict:         yolo predict task=detect model=iti107_assignment\train\weights\best_int8_openvino_model imgsz=640 int8 
Validate:        yolo val task=detect model=iti107_assignment\train\weights\best_int8_openvino_model imgsz=640 data=datasets/data.yaml int8 
Visualize:       https://netron.app


## Inference

Let's test our model on some sample pictures. You can optionally specify the confidence threshold (e.g. `conf=0.5`), and the IoU (e.g. `iou=0.6`) for the NMS. The model will only output the bounding boxes of those detection that exceeds the confidence threshould and the IoU threshold.  

In [13]:
# import ultralytics
# from ultralytics import YOLO
from PIL import Image

# source = 'https://raw.githubusercontent.com/nyp-sit/iti107-2024S2/refs/heads/main/session-3/samples/sample_balloon.jpeg'
source = './test_pic5.png'
model = YOLO("iti107_assignment/train/weights/best_int8_openvino_model", task='detect')
result = model(source, conf=0.4, iou=0.4)

# Visualize the results
for i, r in enumerate(result):
    print(r)
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Show results to screen (in supported environments)
    r.show()

    # Save results to disk
    r.save(filename=f"results{i}.png")

Loading iti107_assignment\train\weights\best_int8_openvino_model for OpenVINO inference...
Using OpenVINO LATENCY mode for batch=1 inference...

image 1/1 e:\Anaconda\envs\00.NYP course\itit107-spdc2\assignment-iti107\test_pic5.png: 640x640 4 fire extinguishers, 2 fire signages, 1 helmet, 40.0ms
Speed: 8.9ms preprocess, 40.0ms inference, 7.4ms postprocess per image at shape (1, 3, 640, 640)
ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'extinguisher-helmet', 1: 'fire extinguisher', 2: 'fire signage', 3: 'helmet'}
obb: None
orig_img: array([[[ 98, 101, 107],
        [ 98, 100, 108],
        [ 98, 100, 109],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[ 97, 100, 106],
        [ 96,  99, 106],
        [ 96,  98, 107],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[ 96,  98, 104],
      